In [1]:
from PIL import  ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


## Getting the list of files
from pathlib import Path

fish_dir = Path('./fishes/Fish_Dataset/Fish_Dataset')
file_path = list(fish_dir.glob(r"**/*.png"))

file_path[:10]

[WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00001.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00002.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00003.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00004.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00005.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00006.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00007.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00008.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00009.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00010.png')]

In [2]:
## Getting the labels of the images
from os import path
import pandas as pd

labels = list(map(lambda f: path.split(path.split(f)[0])[1], file_path))

labels = pd.Series(labels)
labels.unique()

array(['Black Sea Sprat', 'Black Sea Sprat GT', 'Gilt-Head Bream',
       'Gilt-Head Bream GT', 'Hourse Mackerel', 'Hourse Mackerel GT',
       'Red Mullet', 'Red Mullet GT', 'Red Sea Bream', 'Red Sea Bream GT',
       'Sea Bass', 'Sea Bass GT', 'Shrimp', 'Shrimp GT',
       'Striped Red Mullet', 'Striped Red Mullet GT', 'Trout', 'Trout GT'],
      dtype=object)

In [3]:
## Images series
images = pd.Series(file_path).astype(str)
images.head()

0    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
1    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
2    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
3    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
4    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
dtype: object

In [4]:
## Creating dataframe of fishes with their labels
fishes = pd.concat([images, labels], axis=1)
fishes.columns = ["image", "label"]
fishes = fishes[fishes["label"].apply(lambda x: x[-2:] != "GT")].sample(frac=1).reset_index(drop=True)
fishes.shape

(9000, 2)

In [5]:
## Splitting data
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(fishes, test_size=0.1, random_state=15, shuffle=True)

print(train_data.shape)
print(test_data.shape)

(8100, 2)
(900, 2)


In [6]:
print(train_data.label.value_counts())
print("###")
print(test_data.label.value_counts())

Sea Bass              916
Gilt-Head Bream       907
Shrimp                905
Red Sea Bream         902
Black Sea Sprat       897
Red Mullet            897
Hourse Mackerel       895
Striped Red Mullet    892
Trout                 889
Name: label, dtype: int64
###
Trout                 111
Striped Red Mullet    108
Hourse Mackerel       105
Black Sea Sprat       103
Red Mullet            103
Red Sea Bream          98
Shrimp                 95
Gilt-Head Bream        93
Sea Bass               84
Name: label, dtype: int64


In [7]:
## Image Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
input_shape = (224, 224, 3)

train_gen = ImageDataGenerator(rescale=1. / 255, validation_split=0.15)
test_gen = ImageDataGenerator()

train_images = train_gen.flow_from_dataframe(dataframe=train_data,
                                             x_col="image",
                                             y_col="label",
                                             target_size=img_size,
                                             class_mode="categorical",
                                             batch_size=32,
                                             subset="training",
                                             seed=15,
                                             color_mode="rgb",
                                             shuffle=True)

validation_images = train_gen.flow_from_dataframe(dataframe=train_data,
                                                  x_col="image",
                                                  y_col="label",
                                                  target_size=img_size,
                                                  class_mode="categorical",
                                                  batch_size=32,
                                                  subset="validation",
                                                  seed=15,
                                                  color_mode="rgb",
                                                  shuffle=True)

test_images = test_gen.flow_from_dataframe(dataframe=test_data,
                                           x_col="image",
                                           y_col="label",
                                           target_size=img_size,
                                           class_mode="categorical",
                                           batch_size=32,
                                           color_mode="rgb",
                                           shuffle=False)

Found 6885 validated image filenames belonging to 9 classes.
Found 1215 validated image filenames belonging to 9 classes.
Found 900 validated image filenames belonging to 9 classes.


In [8]:
from keras import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten, Dense, Conv2D

model = Sequential([Conv2D(32, (5, 5), activation='relu', strides=(1, 1), input_shape=input_shape, padding="same"),
                    MaxPooling2D(pool_size=(2, 2)), Flatten(),
                    Dense(128, activation='relu'), Dropout(0.15),
                    Dense(128, activation='relu'), Dropout(0.15), Dense(9, activation="softmax")])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 401408)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               51380352  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger

rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", patience=5, verbose=1, factor=0.2, min_lr=0.0001)
early_stop = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)
csv_logger = CSVLogger("model_history_log.csv", append=True)

In [11]:
history = model.fit(train_images, validation_data=validation_images, epochs=50, verbose=1,
                    callbacks=[rate_reduction, early_stop, csv_logger])

Epoch 1/50
216/216 [==============================] - 129s 599ms/step - loss: 0.9478 - accuracy: 0.7275 - val_loss: 0.1994 - val_accuracy: 0.9309
Epoch 2/50
216/216 [==============================] - 128s 593ms/step - loss: 0.1305 - accuracy: 0.9589 - val_loss: 0.1041 - val_accuracy: 0.9638
Epoch 3/50
216/216 [==============================] - 128s 594ms/step - loss: 0.0545 - accuracy: 0.9832 - val_loss: 0.1014 - val_accuracy: 0.9679
Epoch 4/50
216/216 [==============================] - 130s 601ms/step - loss: 0.0314 - accuracy: 0.9904 - val_loss: 0.0839 - val_accuracy: 0.9778
Epoch 5/50
216/216 [==============================] - 129s 596ms/step - loss: 0.0261 - accuracy: 0.9922 - val_loss: 0.1319 - val_accuracy: 0.9621
Epoch 6/50
216/216 [==============================] - 128s 593ms/step - loss: 0.0185 - accuracy: 0.9942 - val_loss: 0.0527 - val_accuracy: 0.9852
Epoch 7/50
216/216 [==============================] - 127s 590ms/step - loss: 0.0219 - accuracy: 0.9922 - val_loss: 0.1073 -

In [12]:
import numpy as np

prediction = np.argmax(model.predict(test_images), axis=1)
predicted_labels = test_images.class_indices
predicted_labels = dict((v, k) for k, v in predicted_labels.items())
predictions = [predicted_labels[k] for k in prediction]

In [13]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_test = test_data.label
print(classification_report(y_test, predictions))

print(accuracy_score(y_test, predictions))
confusion_matrix(y_test, predictions, normalize='true')

                    precision    recall  f1-score   support

   Black Sea Sprat       0.97      0.99      0.98       103
   Gilt-Head Bream       0.98      1.00      0.99        93
   Hourse Mackerel       0.99      0.98      0.99       105
        Red Mullet       1.00      0.96      0.98       103
     Red Sea Bream       0.95      0.98      0.96        98
          Sea Bass       0.95      0.99      0.97        84
            Shrimp       0.98      0.98      0.98        95
Striped Red Mullet       1.00      0.95      0.98       108
             Trout       0.99      0.99      0.99       111

          accuracy                           0.98       900
         macro avg       0.98      0.98      0.98       900
      weighted avg       0.98      0.98      0.98       900

0.98


array([[0.99029126, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00970874],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.98095238, 0.        , 0.        ,
        0.        , 0.01904762, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.96116505, 0.00970874,
        0.02912621, 0.        , 0.        , 0.        ],
       [0.01020408, 0.        , 0.        , 0.        , 0.97959184,
        0.01020408, 0.        , 0.        , 0.        ],
       [0.        , 0.01190476, 0.        , 0.        , 0.        ,
        0.98809524, 0.        , 0.        , 0.        ],
       [0.02105263, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.97894737, 0.        , 0.        ],
       [0.        , 0.        , 0.00925926, 0.        , 0.03703704,
        0.        , 0.        , 0.9537037 , 0.        ],


In [14]:
from tensorflowjs import converters

converters.save_keras_model(model, "tf-js-model")

print("Saved successfully")

Saved successfully


In [15]:
print(test_images.filenames[:6])
print(predictions[:6])

['fishes\\Fish_Dataset\\Fish_Dataset\\Red Sea Bream\\Red Sea Bream\\00580.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Shrimp\\Shrimp\\00438.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Hourse Mackerel\\Hourse Mackerel\\00976.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Sea Bass\\Sea Bass\\00345.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Black Sea Sprat\\Black Sea Sprat\\00719.png', 'fishes\\Fish_Dataset\\Fish_Dataset\\Gilt-Head Bream\\Gilt-Head Bream\\00361.png']
['Red Sea Bream', 'Shrimp', 'Hourse Mackerel', 'Sea Bass', 'Black Sea Sprat', 'Gilt-Head Bream']
